In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
cx=pd.read_csv('/home/andre301267/git/BD/temp/imp_fert_ncm_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])

In [22]:
cx.drop(columns=['CO_NCM','VL_FOB_orig','VL_FRETE_orig','VL_SEGURO_orig','VL_SEGURO'],inplace=True)

In [23]:
nick=['MOP']
km=[2]
UF=['BA']
CO_PAIS=[]
CO_URF=[]

In [24]:
cat_list=['nick','km','UF','CO_PAIS','CO_URF']
cat_dict={'nick':nick,'km':km,'UF':UF,'CO_PAIS':CO_PAIS,'CO_URF':CO_URF}

In [25]:
for col in cat_list:
    if len(cat_dict[col])>0:
        cx=cx[cx[col].isin(cat_dict[col])]
    else:
        cx.drop(columns=[col],inplace=True)

In [26]:
col_cat=[x for x in ['nick','km','UF','CO_PAIS','CO_URF'] if x in cx.columns]

col_quanti=['KG_LIQUIDO','VL_FOB','VL_FRETE']

cx=cx.groupby(by=['CO_ANO','CO_MES']+col_cat)[col_quanti].sum().reset_index()

In [37]:
%run '/home/andre301267/git/BD/Comexstat/function/matriz_esparsa.ipynb'

** Função matriz_esparsa(comex_df,col_cat) disponível **


In [38]:
cx=matriz_esparsa(cx,col_cat)

KeyError: "['KG_LIQUIDO', 'VL_FOB', 'VL_FRETE'] not in index"

In [36]:
cx

,CO_ANO,CO_MES,date,nick,km,UF,tipo
0,1997,8,1997-08-01,MOP,2,BA,train
1,1997,9,1997-09-01,MOP,2,BA,train
2,1997,10,1997-10-01,MOP,2,BA,train
3,1997,11,1997-11-01,MOP,2,BA,train
4,1997,12,1997-12-01,MOP,2,BA,train
...,...,...,...,...,...,...,...
320,2024,4,2024-04-01,MOP,2,BA,train
321,2024,5,2024-05-01,MOP,2,BA,train
322,2024,6,2024-06-01,MOP,2,BA,train
323,2024,7,2024-07-01,MOP,2,BA,train


In [33]:
cx=cx.assign(tipo='train')

In [ ]:
cx_train=cx[cx.tipo=='train']

In [ ]:
cx_prev=cx[cx.date==cx.date.max()]

In [ ]:
cx_prev.CO_MES=cx_prev.CO_MES+1

In [ ]:
cx_prev.CO_MES.values>0

In [ ]:
if cx_prev.CO_MES.values>12:
    cx_prev.CO_ANO=cx_prev.CO_ANO+1
    cx_prev.CO_MES=1

In [ ]:
cx=pd.concat([cx_train,cx_prev.assign(tipo='prev')])

In [ ]:
cx.date=cx.date.astype(int)

# Previsão

In [ ]:
cx[cx.tipo=='prev'].date=pd.to_datetime(cx[cx.tipo=='prev'].CO_ANO.astype(str)+'/'+cx[cx.tipo=='prev'].CO_MES.astype(str)+'/1').astype(int)

# Calssificação

In [ ]:
# Marca ocorrências
cx=cx.assign(oc=0)
cx.loc[cx.KG_LIQUIDO>0,'oc']=1
cx.loc[cx.tipo=='prev',col_quanti+['oc']]=np.nan
prev=cx[cx.tipo=='prev']
cx.loc[cx.tipo=='prev','date']=pd.to_datetime(prev.CO_ANO.astype(str)+'/'+prev.CO_MES.astype(str)+'/1').astype(int)

In [ ]:
cx[['CO_MES']+col_cat+['tipo']]=cx[['CO_MES']+col_cat+['tipo']].astype('category')

In [ ]:
cx_back=cx.copy()

In [ ]:
cx=pd.get_dummies(cx)

# Classificação

In [ ]:
train=cx[cx.tipo_train==True]

In [ ]:
X_train=train[['date']+train.select_dtypes('bool').columns.tolist()]

In [ ]:
y_train=train.oc

In [ ]:
prev=cx[cx.tipo_prev==True]

In [ ]:
X_prev=prev[['date']+prev.select_dtypes('bool').columns.tolist()]

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
RFC=RFC()

In [ ]:
RFC.fit(X_train,y_train)

In [ ]:
X_prev=X_prev.assign(oc=RFC.predict(X_prev))

# Regressão

In [ ]:
train=train[train.oc==1]

In [ ]:
X_train=train[['date']+train.select_dtypes('bool').columns.tolist()]

In [ ]:
y_train=train[col_quanti]

In [ ]:
X_prev=X_prev[X_prev.oc==1].drop(columns='oc')

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFR
RFR=RFR()

In [ ]:
RFR.fit(X_train,y_train)

In [ ]:
prev=RFR.predict(X_prev)

In [ ]:
cx_back[col_quanti]=list(prev.flat)

In [ ]:
cx_back